In [2]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
import tensorflow as tf
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Load the training set of images and labels
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/SI_flood_dataset/dataset/SI_flood_dataset_split/train_eq',
    image_size=(256, 256),
    batch_size=32,
    label_mode='categorical',
)

# Preprocess the images in the training set
train_dataset = train_dataset.map(
    lambda x, y: (tf.image.per_image_standardization(x), y)
)

# Load the validation set of images and labels
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/SI_flood_dataset/dataset/SI_flood_dataset_split/val_eq',
    image_size=(256, 256),
    batch_size=32,
    label_mode='categorical',
)

# Preprocess the images in the validation set
valid_dataset = valid_dataset.map(
    lambda x, y: (tf.image.per_image_standardization(x), y)
)

# Define the CNN architecture
model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(256, 256, 3),
    pooling='avg'
)

# Extract features from the images in the training set using the trained CNN
train_features = []
train_labels = []
for images, labels in train_dataset:
    features = model.predict(images)
    train_features.append(features)
    train_labels.append(labels)
train_features = np.concatenate(train_features, axis=0)
train_labels = np.concatenate(train_labels, axis=0)

# Convert the 2-dimensional label matrix into a 1-dimensional label vector
train_labels = np.argmax(train_labels, axis=1)

# Train a logistic regression model on the features and labels in the training set
logistic_model = LogisticRegression()
logistic_model.fit(train_features, train_labels)

# Extract features from the images in the validation set using the trained CNN
valid_features = []
valid_labels = []
for images, labels in valid_dataset:
    features = model.predict(images)
    valid_features.append(features)
    valid_labels.append(labels)
valid_features = np.concatenate(valid_features, axis=0)
valid_labels = np.concatenate(valid_labels, axis=0)

# Convert the 2-dimensional label matrix into a 1-dimensional label vector
valid_labels = np.argmax(valid_labels, axis=1)

# Make predictions on the validation set using the logistic regression model
y_pred = logistic_model.predict(valid_features)

# Evaluate the model using accuracy, F1-score, omission, and commission metrics on the validation set
acc = accuracy_score(valid_labels, y_pred)
f1 = f1_score(valid_labels, y_pred, average='weighted')
tn, fp, fn, tp = confusion_matrix(valid_labels, y_pred).ravel()
omission = fn / (fn + tp)
commission = fp / (fp + tn)

print('Accuracy:', acc)
print('F1-score:', f1)
print('Omission:', omission)
print('Commission:', commission)

Found 736 files belonging to 2 classes.
Found 186 files belonging to 2 classes.
1/1 [==============================] - 0s 48ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1/1 [==============================] - 1s 850ms/step
Accuracy: 0.9301075268817204
F1-score: 0.9301055065760948
Omission: 0.06451612903225806
Commission: 0.07526881720430108
